#### Data Preparation for Analysis

12/9/21
Paul R. Phillips

**Description**<br>
This article describes how to join the Twitter *crypto indicator* attribute to the crypto price movement data. Here we will clean the Twitter data's *create_at* attribute in order to fit with the Yahoo! Finance data's daily-level granularity. Finally, we will illustrate how to join the two data sets together

### Configure Notebook

In [ ]:
## Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt

## Functions

### Import Data

Since the cryptocurrency data is at the daily level, we need to parse out the date value from the *created_at* column. We need to proceed with caution and recognize that Elon may have only tweeted about crypto once for a given day. When we reduce the duplicated date values, we need to ensure we're not missing events. The way to approach this will be to aggregate on the *maximum* value of the event signal for a given date.

##### Import Twitter Data

In [ ]:
# Calculate number of crypto related instances
elon_crypto_tweets = elon_twitter_data.loc[elon_twitter_data.is_crypto_related == 1, :].shape[0]


106

In [ ]:
# Convert timestamp to datetime upon import
elon_twitter_data = pd.read_csv('final_elon_twitter_data.csv',
                                parse_dates=['created_at'])

# Create 'Date' variable & drop 'created_at'
elon_twitter_data['Date'] = elon_twitter_data.created_at.dt.date
elon_twitter_data = elon_twitter_data.drop(columns='created_at')

# Reorder columns
elon_twitter_data = elon_twitter_data.reindex(columns=['Date', 'is_crypto_related'])

# Reconvert 'Date' to datetime?
elon_twitter_data['Date'] = pd.to_datetime(elon_twitter_data['Date'], format='%Y-%m-%d')

# Calculate number of crypto related instances
elon_crypto_tweets = elon_twitter_data.loc[elon_twitter_data.is_crypto_related == 1, :].shape[0]

# Aggregate date column
elon_twitter_data_daily = elon_twitter_data.groupby('Date', as_index=False)['is_crypto_related'].sum()
elon_twitter_data_daily_total = elon_twitter_data.groupby('Date', as_index=False)['is_crypto_related'].count()

elon_twitter_data_daily = elon_twitter_data_daily.merge(elon_twitter_data_daily_total,
                                                        on='Date',
                                                        how='left')

# Filter dates greater than 11/28/21 -- max range for coins
elon_twitter_data_daily = elon_twitter_data_daily.loc[elon_twitter_data.Date <= '2021-11-28', :].rename(columns={'is_crypto_related_x':'crypto_related_tweet',
                                                                                                                'is_crypto_related_y':'total_tweets'})

# Write out prepared data
#elon_twitter_data.to_csv('final_elon_twitter_data_daily.csv',
#                          index=False,
#                          encoding='utf-8')

elon_twitter_data_daily['crypto_related_weight'] = elon_twitter_data_daily['crypto_related_tweet']/elon_twitter_data_daily['total_tweets']

# Display first few rows
elon_twitter_data_daily.loc[elon_twitter_data_daily.crypto_related_tweet != 0, :].head()


Date  crypto_related_tweet  total_tweets  crypto_related_weight
8  2020-12-03                     1             4               0.250000
22 2020-12-20                     3            17               0.176471
24 2020-12-22                     1             3               0.333333
43 2021-01-11                     1             8               0.125000
53 2021-01-22                     1             4               0.250000

##### Import Shib Data

In [ ]:
# Import SHIB coin data

shib_coin_data = pd.read_csv('shiba-inu_2020-11-15_2021-11-28.csv',
                             parse_dates=['Date'])

# Sort in ascending order by 'Date'
shib_coin_data = shib_coin_data.sort_values(by='Date', ignore_index=True)

shib_coin_data.head()

Date          Open          High  ...         Close     Volume  Market Cap
0 2020-11-15  6.046432e-11  6.046432e-11  ...  5.830975e-11  66.237829         0.0
1 2020-11-16  5.830975e-11  6.045499e-11  ...  6.017378e-11  14.900985         0.0
2 2020-11-17  6.017378e-11  6.290416e-11  ...  6.266734e-11  33.161162         0.0
3 2020-11-18  6.266734e-11  6.270988e-11  ...  6.270988e-11   8.819783         0.0
4 2020-11-19  6.270988e-11  6.270988e-11  ...  6.156925e-11   9.539596         0.0

[5 rows x 7 columns]

##### Join Twitter Data to Crypto Currency

In [ ]:
# Join Dataframes
shib_twitter_data = shib_coin_data.merge(elon_twitter_data_daily,
                                         how='left',
                                         on='Date').rename(columns={'Date_x':'Date',
                                                                         'Close':'shib_close',
                                                                         'Volume':'shib_volume'})
# Fill NA's with 0
shib_twitter_data['crypto_related_tweet'] = shib_twitter_data.crypto_related_tweet.fillna(0).astype(int)
shib_twitter_data['crypto_related_weight'] = shib_twitter_data.crypto_related_weight.fillna(0)

# Filter out dates prior to November 24th (latest date we have for Twitter events)
# Keep only columns we care about
shib_twitter_data = shib_twitter_data.loc[shib_twitter_data.Date >= '2020-11-24', ['Date',
                                                                                   'crypto_related_tweet',
                                                                                   'crypto_related_weight',
                                                                                   'shib_close',
                                                                                   'shib_volume']]

shib_twitter_data.head()

Date  crypto_related_tweet  ...    shib_close  shib_volume
9  2020-11-24                     0  ...  6.846712e-11    78.137549
10 2020-11-25                     0  ...  6.663524e-11    33.833382
11 2020-11-26                     0  ...  6.663524e-11     8.791897
12 2020-11-27                     0  ...  6.663524e-11     8.791897
13 2020-11-28                     0  ...  5.837675e-11   887.871907

[5 rows x 5 columns]

##### Join the Rest of the Crypto Coin Datasets

In [ ]:
# Import other coin sets
bitcoin_data = pd.read_csv('BTC-USD.csv',
                           usecols=['Date',
                                  'Close',
                                  'Volume'],
                           parse_dates=['Date']).rename(columns={'Close':'bitcoin_close',
                                                                 'Volume':'bitcoin_volume'})
ethereum_data = pd.read_csv('ETH-USD.csv',
                           usecols=['Date',
                                  'Close',
                                  'Volume'],
                           parse_dates=['Date']).rename(columns={'Close':'ethereum_close',
                                                                 'Volume':'ethereum_volume'})
dogecoin_data = pd.read_csv('DOGE-USD.csv',
                           usecols=['Date',
                                  'Close',
                                  'Volume'],
                           parse_dates=['Date']).rename(columns={'Close':'dogecoin_close',
                                                                 'Volume':'dogecoin_volume'})
                           
# Sort in ascending order by 'Date'
bitcoin_data = bitcoin_data.sort_values(by='Date', ignore_index=True)
ethereum_data = ethereum_data.sort_values(by='Date', ignore_index=True)
dogecoin_data = dogecoin_data.sort_values(by='Date', ignore_index=True)

# Join Dataframes 
shib_twitter_data = shib_twitter_data.merge(bitcoin_data,
                                         how='left',
                                         on='Date')
shib_twitter_data = shib_twitter_data.merge(ethereum_data,
                                         how='left',
                                         on='Date')
shib_twitter_data = shib_twitter_data.merge(dogecoin_data,
                                         how='left',
                                         on='Date')

shib_twitter_data.head()

Date  crypto_related_tweet  ...  dogecoin_close  dogecoin_volume
0 2020-11-24                     0  ...        0.004204        266716036
1 2020-11-25                     0  ...        0.003727        174459927
2 2020-11-26                     0  ...        0.003250        175520294
3 2020-11-27                     0  ...        0.003275         92323023
4 2020-11-28                     0  ...        0.003405         58033650

[5 rows x 11 columns]

In [ ]:
shib_twitter_data.to_csv('group5_clean_data_weighted_20211215.csv',
                         index=False, 
                         encoding='utf-8')